In [1]:
#Import the necessary modules
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.wcs import WCS
import warnings
import numpy as np
warnings.filterwarnings('ignore')
from astroquery.vizier import Vizier
from astroquery.gaia import Gaia
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.core.display import HTML
import pandas as pd
from astroquery.simbad import Simbad

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
## change row limit to none; else default to 50 
Vizier.ROW_LIMIT = -1
Gaia.ROW_LIMIT = -1

In [3]:
# inputs
#print("input search values here:")
# scoords = input("Input_RA_DEC_.")
scoords = "11 02 24.8763629208 -77 33 35.667131796" #hour angle
# srad = input("Input_Search_Radius.")
srad = 0.5 #deg

def query_the_gaia(objloc,conerad,catalognamelist=["I/350/gaiaedr3","B/wds"],RUWE=True,maghigh=3,maglow=10):
    """
    This function will query the specified catalogs using Astropy Vizier.

    Args:
        catalognamelist (list): List of catalog name strings as shown on Vizier. 
        Ex. "I/350/gaiaedr3"
        objra (float): RA coord in degrees
        objdec (float): Dec coord in degrees
        conerad (float): Cone radius in degrees

    Returns:
        _type_: _description_
    """    
    if len(objloc) == 2:
         obj_coord=SkyCoord(objloc[0],objloc[1],unit=(u.degree, u.degree), frame='icrs')
    else:
        obj_coord=SkyCoord(objloc,unit=(u.hourangle, u.degree), frame='icrs')
    result = Vizier.query_region(obj_coord,
                            radius=u.Quantity(conerad,u.deg),
                             catalog=catalognamelist )
    
    
    if RUWE:
        result=result[0][result[0]['RUWE']<1.2]

    #filtering more by G magnitude
    result=result[result['Gmag']>maghigh] 
    result=result[result['Gmag']<maglow]
        
    gaia_id_list=result['Source']

    # TODO add column headers for name, RA, DEC, RUWE, mag
    singles=[]
    for i in gaia_id_list:
        id= "Gaia DR3"+str(i)
        info=Simbad.query_objectids(id)
        strinfo=str(info)
        if 'wds' in strinfo:
            gaia_id_list.remove(i)
        else:
            singles.append(info[0])

    return singles

temp_table = query_the_gaia(objloc=scoords,conerad=srad,catalognamelist=["I/355/gaiadr3"])
#print(temp_table)

In [4]:
pd.DataFrame(temp_table)

,0
0,"[[Gaia DR3 5201001680866216704], [TIC 44994436..."
1,"[[Gaia DR3 5201122936381243392], [TIC 45432832..."
2,"[[Gaia DR3 5201184921349846144], [TIC 45422712..."
3,"[[Gaia DR3 5201225740718586624], [TIC 27767117..."
